In [4]:
from isochrones import get_ichrone

iso = get_ichrone('mist')
track = get_ichrone('mist', tracks=True)

In [5]:
from isochrones.starmodel import StarModel

mod = StarModel(iso, Teff=(5700, 50), logg=(4.5, 0.1), feh=(0.01, 0.06), J=(10.3, 0.02), K=(10.0, 0.2), BP=(11, 0.01))

In [6]:
pars = (300, 9.7, 0.0, 300, 0.1)
mod.lnprior(pars)

-24.213148884646188

In [7]:
mod.lnlike(pars, use_cache=False)

-47814.742670380554

In [9]:
%timeit mod.lnlike(pars, use_cache=False)

10000 loops, best of 3: 123 µs per loop


In [7]:
%load_ext line_profiler

In [14]:
from isochrones.observation import ObservationTree, ObsNode

%lprun -f StarModel.lnlike mod.lnlike(pars, use_cache=False)

Timer unit: 1e-06 s

Total time: 0.000378 s
File: /Users/tdm/repositories/isochrones/isochrones/starmodel.py
Function: lnlike at line 507

Line #      Hits         Time  Per Hit   % Time  Line Contents
   507                                               def lnlike(self, p, **kwargs):
   508         1         33.0     33.0      8.7          pardict = self.obs.p2pardict(p)
   509                                           
   510         1          2.0      2.0      0.5          model_values = {}
   511         2          3.0      1.5      0.8          for star, pars in pardict.items():
   512         1        190.0    190.0     50.3              Teff, logg, feh, mags = self.ic.interp_mag(pars, self.bands)
   513         1          1.0      1.0      0.3              vals = {'Teff': Teff,
   514         1          1.0      1.0      0.3                      'logg': logg,
   515         1          1.0      1.0      0.3                      'feh': feh}
   516         1          8.0      8.0 

In [ ]:
n = mod.obs.get_obs_nodes()[0]

In [33]:
m = n.leaves[0]

In [34]:
m.label

'0_0'

In [35]:
m.

AttributeError: 'ModelNode' object has no attribute 'band'

In [30]:
mod.obs.p2pardict(pars)

{'0_0': [300, 9.7, 0.0, 300, 0.1]}

In [26]:
bands = {n.band for n in mod.obs.get_obs_nodes()}
props = {k for v in mod.obs.spectroscopy.values() for k in v.keys()}

In [28]:
props - {'Teff', 'logg', 'feh'}

set()

In [23]:
mod.obs.spectroscopy

{'0_0': {'Teff': (5700, 50), 'logg': (4.5, 0.1)}}

{'Teff', 'logg'}

In [5]:
iso.fehs

AttributeError: 'MIST_Isochrone' object has no attribute 'fehs'

In [18]:
eeps = [300., 350.]

In [19]:
masses, dm_deeps = zip(*[iso.interp_value([eep, 9.5, -0.32], ['initial_mass', 'dm_deep']) for eep in eeps])

In [20]:
masses

(0.6033972313086698, 1.0139212265028263)

In [21]:
dm_deeps

(-0.00015603156852549426, 0.0014208742570490662)

In [5]:
x = [(i, i**2) for i in range(5)]

In [15]:
a, b = zip(*[(i, i**2) for i in range(5)])

In [16]:
a

(0, 1, 2, 3, 4)

In [17]:
b

(0, 1, 4, 9, 16)

In [9]:
import numpy as np
np.vstack(x)

array([[ 0,  0],
       [ 1,  1],
       [ 2,  4],
       [ 3,  9],
       [ 4, 16]])

In [2]:
import numpy as np

pars = np.array([300., 9.5, -0.32, 400, 0.2])
# pars = [np.arange(250, 260, 0.5), 9.5, -0.32, 400., 0.2]
iso.interp_value(pars, ['initial_mass', 'dm_deep'])

/Users/tdm/repositories/isochrones/isochrones/mist/models.py:42: RuntimeWarning: divide by zero encountered in log10
  df['feh'] = df['log_surf_z'] - np.log10(df['surface_h1']) - np.log10(0.0181)  # Aaron Dotter says
/Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


array([ 6.03397231e-01, -1.56031569e-04])

In [2]:
import numpy as np

pars = np.array([300., 9.5, -0.32, 400, 0.2])
# pars = [np.arange(250, 260, 0.5), 9.5, -0.32, 400., 0.2]
iso.interp_value(pars, ['initial_mass', 'dm_deep'])

/Users/tdm/repositories/isochrones/isochrones/mist/models.py:42: RuntimeWarning: divide by zero encountered in log10
  df['feh'] = df['log_surf_z'] - np.log10(df['surface_h1']) - np.log10(0.0181)  # Aaron Dotter says
/Users/tdm/miniconda3/envs/isochrones/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


array([ 6.03397231e-01, -1.56031569e-04])

In [29]:
dict((('a', 1), ('b', 2)))

{'a': 1, 'b': 2}

In [10]:
iso.interp_mag(pars, ['G', 'BP', 'RP'])

(4327.112330581007,
 4.696157851252527,
 -0.30820097729121776,
 array([8.09868503, 8.7955046 , 7.30504895]))

In [11]:
%timeit iso.interp_mag(pars, ['G', 'BP', 'RP'])

10000 loops, best of 3: 43.7 µs per loop


In [4]:
%timeit iso.interp_mag(pars, ['G', 'BP', 'RP'])

The slowest run took 4.94 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 263 µs per loop


In [8]:
263 / len(pars[0])

13.15

In [6]:
%timeit iso.interp_value(pars, ['radius'])
%timeit iso.interp_mag(pars, ['G', 'BP', 'RP'])

The slowest run took 6.26 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.5 µs per loop
10000 loops, best of 3: 40.2 µs per loop


In [14]:
pars2 = np.array([0.67, 300, -0.23, 500, 0.6])
pars2 = [np.arange(0.5, 1.0, 0.05), 300, -0.23, 500, 0.6]
track.interp_value(pars2, ['radius'])

array([[0.47348708],
       [0.52528196],
       [0.57510442],
       [0.61745853],
       [0.65225259],
       [0.68787431],
       [0.72669778],
       [0.76969601],
       [0.81714689],
       [0.86869549]])

In [15]:
track.interp_mag(pars2, ['G', 'BP', 'RP'])

(array([3893.81844253, 4045.51151663, 4234.92425267, 4472.52351642,
        4744.42251718, 5008.6848856 , 5253.91276537, 5479.11070963,
        5685.5497841 , 5874.40749956]),
 array([4.78649349, 4.73771749, 4.69679525, 4.66983547, 4.65440739,
        4.63818249, 4.61851914, 4.59491573, 4.56777767, 4.53816517]),
 array([-0.21686775, -0.21701548, -0.2160241 , -0.21434294, -0.21223029,
        -0.21041783, -0.20930838, -0.2089884 , -0.20924481, -0.20993076]),
 array([[ 9.16135938, 10.16150747,  8.20113619],
        [ 9.09469835, 10.01543052,  8.17281349],
        [ 9.02165596,  9.85315172,  8.14668682],
        [ 8.93453871,  9.6693252 ,  8.11566345],
        [ 8.85252538,  9.49952819,  8.08939251],
        [ 8.79650373,  9.3769123 ,  8.07967222],
        [ 8.76193961,  9.2907239 ,  8.08331592],
        [ 8.74352878,  9.23054526,  8.09755396],
        [ 8.73515497,  9.18739378,  8.11785487],
        [ 8.73240765,  9.15519738,  8.14059441]]))

In [16]:
%timeit track.interp_value(pars2, ['radius'])
%timeit track.interp_mag(pars2, ['G', 'BP', 'RP'])

10000 loops, best of 3: 71.3 µs per loop
10000 loops, best of 3: 192 µs per loop


In [150]:
from math import pi, log, sqrt
LOG_ONE_OVER_ROOT_2PI = log(1./sqrt(2*pi))

@jit(nopython=True)
def star_lnlike(pars, index_order,
                spec_vals, spec_uncs,
                mag_vals, mag_uncs, i_mags,
                model_grid, i_Teff, i_logg, i_feh, i_Mbol, 
                model_ii0, model_ii1, model_ii2, 
                bc_grid, bc_ii0, bc_ii1, bc_ii2, bc_ii3):

    Teff, logg, mags = interp_mag(pars, index_order, model_grid, 
                                  i_Teff, i_logg, i_feh, i_Mbol, 
                                  model_ii0, model_ii1, model_ii2, 
                                  bc_grid, i_mags, bc_ii0, bc_ii1, bc_ii2, bc_ii3)
    
    lnlike = 0
    
    # Spec_vals are Teff, logg, feh
    val = spec_vals[0]
    unc = spec_uncs[0]
    model_val = Teff
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
    
    # logg
    val = spec_vals[1]
    unc = spec_uncs[1]
    model_val = logg
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
        
    # feh
    val = spec_vals[2]
    unc = spec_uncs[2]
    model_val = pars[2]  # feh
    resid = (val - model_val)
    lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
    
    for i in range(len(mag_vals)):
        val = mag_vals[i]
        unc = mag_uncs[i]
        model_val = mags[i]
        lnlike += LOG_ONE_OVER_ROOT_2PI + log(unc) - 0.5 * resid * resid / (unc * unc)
        
    return lnlike




In [151]:
ic = MISTIsochrone()

In [152]:
pars = np.array([300., 9.5, 0.1, 200, 0.2])
bands = ['G', 'BP', 'RP']
ic.interp_mag(pars, bands)

(4185.087791021258,
 4.660144730008982,
 array([7.11836734, 7.89965972, 6.27215866]))

In [153]:
%timeit ic.interp_value(pars, ['logg', 'Teff'])
%timeit ic.interp_mag(pars, bands)

The slowest run took 6.41 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.8 µs per loop
10000 loops, best of 3: 40.6 µs per loop


In [154]:
ic2 = MISTIsochrone_Track()

In [155]:
pars2 = np.array([1.01, 353, 0.11, 200, 0.2])
ic2.interp_mag(pars2, bands)

(5733.612979290512,
 4.424850980556096,
 array([6.69414488, 7.06039213, 6.16097197]))

In [156]:
%timeit ic2.interp_value(pars2, ['logg', 'Teff', 'feh'])
%timeit ic2.interp_mag(pars2, bands)

The slowest run took 6.02 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.8 µs per loop
10000 loops, best of 3: 39.6 µs per loop


In [42]:
from isochrones import StarModel
from isochrones.priors import (salpeter_prior, feh_prior, q_prior,
                               age_prior, distance_prior, AV_prior, 
                               FlatPrior)

class BasicStarModel(StarModel):
#     _not_a_band = ('RA','dec','ra','Dec','maxAV','parallax','AV',
#                   'logg','Teff','feh','density', 'separation',
#                   'PA','resolution','relative','N','index', 'id')
    
    def __init__(self, ic, **kwargs):
        self._ic = ic
        self.kwargs = kwargs

        self.bands = [k for k in kwargs if k in self.ic.bc_grid.bands]
        self.props = [k for k in kwargs if k in self._not_a_band]
                
        self.spec_props = [self.kwargs[k] for k in ['Teff', 'logg', 'feh']]
        
        self._priors = {'mass':salpeter_prior,
                        'feh':feh_prior,
                        'q':q_prior,
                        'age':age_prior,
                        'distance':distance_prior,
                        'AV':AV_prior,
                        'eep':FlatPrior(bounds=(self.ic.mineep, self.ic.maxeep))}

        self._bounds = {'mass':None,
                        'feh':None,
                        'age':None,
                        'q':q_prior.bounds,
                        'distance':distance_prior.bounds,
                        'AV':AV_prior.bounds,
                        'eep':(self.ic.mineep, self.ic.maxeep)}

        if 'maxAV' in kwargs:
            self.set_bounds(AV=(0, kwargs['maxAV']))

        if 'max_distance' in kwargs:
            self.set_bounds(distance=(0, kwargs['max_distance']))
            
    def bounds(self, prop):
        if self._bounds[prop] is not None:
            return self._bounds[prop]
        elif prop=='mass':
            lo, hi = (self.ic.minmass, self.ic.maxmass)
            self._bounds['mass'] = (lo, hi)
            self._priors['mass'].bounds = (lo, hi)
        elif prop=='feh':
            lo, hi = (self.ic.minfeh, self.ic.maxfeh)
            self._bounds['feh'] = (lo, hi)
            self._priors['feh'].bounds = (lo, hi)
        elif prop=='age':
            lo, hi = (self.ic.minage, self.ic.maxage)
            self._bounds['age'] = (lo, hi)
            self._priors['age'].bounds = (lo, hi)
            self._bounds['age'] = (self.ic.minage,
                                   self.ic.maxage)
        else:
            raise ValueError('Unknown property {}'.format(prop))
        return self._bounds[prop]

    def set_bounds(self, **kwargs):
        for k,v in kwargs.items():
            if len(v) != 2:
                raise ValueError('Must provide (min, max)')
            self._bounds[k] = v
            self._priors[k].bounds = v

    def lnlike(self, pars):
        spec_vals = [self.kwargs[prop][0] for prop in ['Teff', 'logg', 'feh']]
        spec_uncs = [self.kwargs[prop][1] for prop in ['Teff', 'logg', 'feh']]
        mag_vals = [self.kwargs[b][0] for b in self.bands]
        mag_uncs = [self.kwargs[b][1] for b in self.bands]
        i_mags = [self.ic.bc_grid.interp.column_index[b] for b in self.bands]
        
        return star_lnlike(pars, spec_vals, spec_uncs,
                           mag_vals, mag_uncs, i_mags, 
                           self.ic.model_grid.interp.grid, 10, 12, 7,
                           *self.ic.model_grid.interp.index_columns,
                           self.ic.bc_grid.interp.grid, 
                           *self.ic.bc_grid.interp.index_columns)
    
    def lnprior(self, pars):
        lnp = 0
        
        # Get prior for EEP:  p(EEP) = p(M) * d(M)/d(EEP)

In [51]:
%load_ext line_profiler

In [56]:
%lprun -f MISTIsochrone.interp_value ic2.interp_value(pars2, ['logg', 'Teff', 'feh'])

Timer unit: 1e-06 s

Total time: 0.001681 s
File: <ipython-input-40-3ad5a92c2310>
Function: interp_value at line 27

Line #      Hits         Time  Per Hit   % Time  Line Contents
    27                                               def interp_value(self, pars, props):
    28                                                   """
    29                                                   
    30                                                   pars : age, feh, eep, [distance, AV]
    31                                                   """
    32         1         86.0     86.0      5.1          pars = np.atleast_1d(pars[self.param_index_order])
    33         1       1595.0   1595.0     94.9          return self.model_grid.interp(pars, props)

In [47]:
%timeit ic2.interp_value(pars2, ['logg', 'Teff'])

The slowest run took 5.22 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12 µs per loop


In [57]:
mod = BasicStarModel(ic, Teff=(5700., 50.), logg=(4.44, 0.1), feh=(0.0, 0.1), G=(9.71, 0.01), BP=(9.85, 0.01))

AttributeError: 'MISTIsochrone' object has no attribute 'mineep'

In [97]:
pars = np.array([300, 8.54, -0.4, 200., 0.1])

In [98]:
mod.lnlike(pars)

-2250.4050015378325

In [99]:
%timeit mod.lnlike(pars)

10000 loops, best of 3: 54.3 µs per loop


In [90]:
mod.lnpost(pars)

AttributeError: 'BasicStarModel' object has no attribute 'obs'

In [ ]:
ic.model_grid.interp.column_index

In [56]:
ic.model_grid.df.columns

Index(['EEP', 'log10_isochrone_age_yr', 'initial_mass', 'star_mass',
       'star_mdot', 'he_core_mass', 'c_core_mass', 'log_L', 'log_LH',
       'log_LHe', 'log_Teff', 'log_R', 'log_g', 'surface_h1', 'surface_he3',
       'surface_he4', 'surface_c12', 'surface_o16', 'log_center_T',
       'log_center_Rho', 'center_gamma', 'center_h1', 'center_he4',
       'center_c12', 'phase', 'feh', 'dm_deep'],
      dtype='object')

In [54]:
pars = np.array([8.54, -0.4, 300., 200., 0.1])

In [55]:
%timeit ic.interp_mag(pars, ['G', 'BP', 'RP', 'J', 'H', 'K'])

The slowest run took 33873.93 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 33.7 µs per loop


In [56]:
ic.bc_grid.bands

['J', 'H', 'K', 'G', 'TESS', 'BP']

In [59]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [61]:
%lprun -f ic.interp_mag ic.interp_mag(np.array([8.54, -0.4, 300., 200., 0.1]), ['G', 'BP'])

Timer unit: 1e-06 s

Total time: 0.00012 s
File: <ipython-input-52-2b6bb4589e2b>
Function: interp_mag at line 32

Line #      Hits         Time  Per Hit   % Time  Line Contents
    32                                               def interp_mag(self, pars, bands):
    33                                                   """
    34                                                   
    35                                                   pars : age, feh, eep, distance, AV
    36                                                   """
    37         1         55.0     55.0     45.8          logTeff, logg, logL = self.model_grid.interp(pars[:3], ['log_Teff', 'log_g', 'log_L'])
    38                                           
    39         1         11.0     11.0      9.2          bc_pars = np.array([10**logTeff, logg, pars[1], pars[4]])  # pars[0] is feh
    40         1         30.0     30.0     25.0          bc = self.bc_grid.interp(bc_pars, bands)        
    41                        

In [63]:
ic.interp_mag(np.array([8.54, -0.4, 300., 200., 0.1]), ['G', 'BP'])

(3.873409202781381, 4.369675032445283, array([9.71631876, 9.85953556]))

In [66]:
from isochrones import StarModel as StarModelOld
from isochrones import get_ichrone

mist = get_ichrone('mist')

mod_old = StarModelOld(mist, Teff=(5700, 50), logg=(4.44, 0.1), G=(9.71, 0.01), BP=(9.85, 0.01))

In [67]:
mod_old.lnlike([400, 9.6, 0.1, 200, 0.1])

KeyError: 'P'

In [49]:
mod.bands

['G']

In [50]:
mod.props

['Teff', 'logg']

In [51]:
mod.kwargs

{'Teff': (5700, 50), 'logg': (4.44, 0.1), 'G': (8.1, 0.01)}

In [5]:
from isochrones.interp import find_indices_3d, find_indices, find_indices_4d

In [6]:
col0 = ic.model_grid.interp.index_columns[0]
col1 = ic.model_grid.interp.index_columns[1]
col2 = ic.model_grid.interp.index_columns[2]
i1, d1, oob1 = find_indices([8.11, -1.1, 300.], ic.model_grid.interp.index_columns)
i2, d2, oob2 = find_indices_3d(8.11, -1.1, 300., col0, col1, col2)
assert (i1 == i2).all()
assert (d1 == d2).all()
assert oob1 == oob2
%timeit find_indices_3d(8.11, -1.1, 300., col0, col1, col2)

The slowest run took 4.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.62 µs per loop


In [7]:
col0 = ic.bc_grid.interp.index_columns[0]
col1 = ic.bc_grid.interp.index_columns[1]
col2 = ic.bc_grid.interp.index_columns[2]
col3 = ic.bc_grid.interp.index_columns[3]
i1, d1, oob1 = find_indices([5842., 4.34, 0.03, 0.42], ic.bc_grid.interp.index_columns)
i2, d2, oob2 = find_indices_4d(5842., 4.34, 0.03, 0.42, col0, col1, col2, col3)
assert (i1 == i2).all()
assert (d1 == d2).all()
assert oob1 == oob2
%timeit find_indices_4d(5842., 4.34, 0.03, 0.42, col0, col1, col2, col3)

The slowest run took 4.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.83 µs per loop


In [8]:
%timeit find_indices(np.array([5842., 4.34, 0.03, 0.42]), ic.bc_grid.interp.index_columns)

The slowest run took 53366.84 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.68 µs per loop


In [9]:
%timeit find_indices(np.array([8.11, -1.1, 300]), ic.model_grid.interp.index_columns)

The slowest run took 49952.27 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.48 µs per loop


In [12]:
%load_ext line_profiler

In [17]:
pars = np.array([8.11, -1.1, 300, 200, 0.])
bands = ['G', 'BP']
%timeit ic.interp_mag(pars, bands)

The slowest run took 5.29 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 28.1 µs per loop


In [16]:
%timeit ic.interp_value(pars, ['log_g', 'log_Teff', 'log_L'])

The slowest run took 6.77 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.88 µs per loop


In [15]:
%lprun -f ic.model_grid.interp.__call__ ic.interp_value(pars, ['log_g', 'log_Teff', 'log_L'])

Timer unit: 1e-06 s

Total time: 5.8e-05 s
File: /Users/tdm/repositories/isochrones/isochrones/interp.py
Function: __call__ at line 441

Line #      Hits         Time  Per Hit   % Time  Line Contents
   441                                               def __call__(self, p, cols):
   442         1         23.0     23.0     39.7          icols = np.array([self.column_index[col] for col in cols])
   443         1          3.0      3.0      5.2          args = (p, self.grid, icols, self.index_columns)
   444                                           
   445         1          2.0      2.0      3.4          if self.ndim == 3:
   446         1          3.0      3.0      5.2              args = (p[0], p[1], p[2], self.grid, icols,
   447         1          1.0      1.0      1.7                      self.index_columns[0], self.index_columns[1],
   448         1          1.0      1.0      1.7                      self.index_columns[2])
   449         1          3.0      3.0      5.2           

In [90]:
pars = np.array([8.11, -1.1, 300, 200, 0.])
bands = ['G', 'BP']
ic.interp_mag(pars, bands)

array([8.64000568, 8.60319518])

In [94]:
%timeit ic.interp_value(pars, ['log_g'])

The slowest run took 5.83 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.96 µs per loop


In [85]:
pars = np.array([8.11, -1.1, 300, 200, 1.1])
ic.interp_mag(pars, bands)

array([9.70095322, 9.85891442])

In [ ]:
ic.interp_value()

In [21]:
mist.mag['G'](300., 8.11, -1.1, 200., 0.)

array([8.64305202])

In [24]:
mist.mag['G'](300., 8.11, -1.1, 200., 1.1)

array([9.51726849])

In [18]:
bc.df.columns

Index(['Bessell_U', 'Bessell_B', 'Bessell_V', 'Bessell_R', 'Bessell_I',
       '2MASS_J', '2MASS_H', '2MASS_Ks', 'Kepler_Kp', 'Kepler_D51',
       'Hipparcos_Hp', 'Tycho_B', 'Tycho_V', 'Gaia_G_DR2Rev', 'Gaia_BP_DR2Rev',
       'Gaia_RP_DR2Rev', 'TESS'],
      dtype='object')

In [40]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [41]:
%lprun -f ic.interp_mag ic.interp_mag(pars, bands)

Timer unit: 1e-06 s

Total time: 0.000136 s
File: <ipython-input-26-365158820d11>
Function: interp_mag at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                               def interp_mag(self, pars, bands):
    15                                                   """
    16                                                   
    17                                                   pars : age, feh, eep, distance, AV
    18                                                   """
    19         1         65.0     65.0     47.8          logTeff, logg, logL = self.models.interp(pars[:3], ['log_Teff', 'log_g', 'log_L'])
    20                                           
    21         1         13.0     13.0      9.6          bc_pars = np.array([10**logTeff, logg, pars[1], pars[4]])  # pars[0] is feh
    22         1         33.0     33.0     24.3          bc = self.bcs.interp(bc_pars, bands)        
    23                               

In [13]:
%timeit grid.interp([8.11, -1.1, 300.], ['log_L', 'initial_mass', 'star_mass', 'log_Teff'])

The slowest run took 4.86 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 15.5 µs per loop


In [84]:
grid.df.head()

EEP  log10_isochrone_age_yr  initial_mass  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 35    35                     5.0      0.100000   
                            36    36                     5.0      0.102885   
                            37    37                     5.0      0.107147   
                            38    38                     5.0      0.111379   
                            39    39                     5.0      0.115581   

                                 star_mass     star_mdot  he_core_mass  \
log10_isochrone_age_yr feh  EEP                                          
5.0                    -4.0 35    0.100000 -1.455094e-13           0.0   
                            36    0.102885 -1.562027e-13           0.0   
                            37    0.107147 -1.707298e-13           0.0   
                            38    0.111379 -1.836256e-13           0.0   
                            39    0.115581 -1.949639e-13           0.0   

                                 c_core_mass     log_L    log_LH  log_LHe  \
log10_isochrone_age_yr feh  EEP                                             
5.0                    -4.0 35           0.0 -0.489734 -1.922517    -99.0   
                            36           0.0 -0.472691 -1.878584    -99.0   
                            37           0.0 -0.447471 -1.814151    -99.0   
                            38           0.0 -0.422498 -1.749734    -99.0   
                            39           0.0 -0.397776 -1.685330    -99.0   

                                   ...      surface_o16  log_center_T  \
log10_isochrone_age_yr feh  EEP    ...                                  
5.0                    -4.0 35     ...     6.108882e-07      5.857643   
                            36     ...     6.108882e-07      5.861047   
                            37     ...     6.108882e-07      5.866071   
                            38     ...     6.108882e-07      5.871107   
                            39     ...     6.108882e-07      5.876158   

                                 log_center_Rho  center_gamma  center_h1  \
log10_isochrone_age_yr feh  EEP                                            
5.0                    -4.0 35        -0.170338      0.299672   0.750981   
                            36        -0.179318      0.295785   0.750981   
                            37        -0.192620      0.290038   0.750981   
                            38        -0.205703      0.284326   0.750981   
                            39        -0.218572      0.278647   0.750981   

                                 center_he4    center_c12  phase  feh  \
log10_isochrone_age_yr feh  EEP                                         
5.0                    -4.0 35     0.248961  2.496535e-07   -1.0 -4.0   
                            36     0.248961  2.496535e-07   -1.0 -4.0   
                            37     0.248961  2.496535e-07   -1.0 -4.0   
                            38     0.248961  2.496535e-07   -1.0 -4.0   
                            39     0.248961  2.496535e-07   -1.0 -4.0   

                                  dm_deep  
log10_isochrone_age_yr feh  EEP            
5.0                    -4.0 35   0.000688  
                            36   0.000681  
                            37   0.000322  
                            38  -0.000029  
                            39  -0.000027  

[5 rows x 27 columns]

In [73]:
len(grid.df)

1328838

In [56]:
df = grid.df_all()

In [57]:
df.head()

EEP  log10_isochrone_age_yr  initial_mass  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 35    35                     5.0      0.100000   
                            36    36                     5.0      0.102885   
                            37    37                     5.0      0.107147   
                            38    38                     5.0      0.111379   
                            39    39                     5.0      0.115581   

                                 star_mass     star_mdot  he_core_mass  \
log10_isochrone_age_yr feh  EEP                                          
5.0                    -4.0 35    0.100000 -1.455094e-13           0.0   
                            36    0.102885 -1.562027e-13           0.0   
                            37    0.107147 -1.707298e-13           0.0   
                            38    0.111379 -1.836256e-13           0.0   
                            39    0.115581 -1.949639e-13           0.0   

                                 c_core_mass     log_L    log_LH  log_LHe  \
log10_isochrone_age_yr feh  EEP                                             
5.0                    -4.0 35           0.0 -0.489734 -1.922517    -99.0   
                            36           0.0 -0.472691 -1.878584    -99.0   
                            37           0.0 -0.447471 -1.814151    -99.0   
                            38           0.0 -0.422498 -1.749734    -99.0   
                            39           0.0 -0.397776 -1.685330    -99.0   

                                ...    surface_c12   surface_o16  \
log10_isochrone_age_yr feh  EEP ...                                
5.0                    -4.0 35  ...   2.496535e-07  6.108882e-07   
                            36  ...   2.496535e-07  6.108882e-07   
                            37  ...   2.496535e-07  6.108882e-07   
                            38  ...   2.496535e-07  6.108882e-07   
                            39  ...   2.496535e-07  6.108882e-07   

                                 log_center_T  log_center_Rho  center_gamma  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 35       5.857643       -0.170338      0.299672   
                            36       5.861047       -0.179318      0.295785   
                            37       5.866071       -0.192620      0.290038   
                            38       5.871107       -0.205703      0.284326   
                            39       5.876158       -0.218572      0.278647   

                                 center_h1  center_he4    center_c12  phase  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 35    0.750981    0.248961  2.496535e-07   -1.0   
                            36    0.750981    0.248961  2.496535e-07   -1.0   
                            37    0.750981    0.248961  2.496535e-07   -1.0   
                            38    0.750981    0.248961  2.496535e-07   -1.0   
                            39    0.750981    0.248961  2.496535e-07   -1.0   

                                 feh  
log10_isochrone_age_yr feh  EEP       
5.0                    -4.0 35  -4.0  
                            36  -4.0  
                            37  -4.0  
                            38  -4.0  
                            39  -4.0  

[5 rows x 26 columns]

In [25]:
len(df)

1328838

In [18]:
grid.get_existing_filenames()

['/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.50_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m0.75_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.00_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.25_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.50_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m1.75_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m2.00_afe_p0.0_vvcrit0.0_basic.iso',
 '/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_basic_isos/MIST_v1.2_feh_m2.50_afe_p0.0_vvcrit0.0_basi

In [11]:
ls /Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_full_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_full.iso

/Users/tdm/.isochrones/mist/MIST_v1.2_vvcrit0.0_full_isos/MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_full.iso


In [14]:
grid = MISTModelGrid()

In [ ]:
df.to

In [23]:
df.head()

,EEP,log10_isochrone_age_yr,initial_mass,star_mass,star_mdot,he_core_mass,c_core_mass,o_core_mass,log_L,log_L_div_Ledd,...,nu_max,acoustic_cutoff,max_conv_vel_div_csound,max_gradT_div_grada,gradT_excess_alpha,min_Pgas_div_P,max_L_rad_div_Ledd,e_thermal,phase,feh
0,15,5.0,0.100000,0.100000,-9.296476e-14,0.0,0.0,0.0,-0.793690,-4.301081,...,210.053677,1426.188700,0.139220,1.105393,0.0,0.999989,0.000017,2.444569e+46,-1.0,-0.25
1,16,5.0,0.104585,0.104585,-1.045561e-13,0.0,0.0,0.0,-0.761784,-4.244876,...,207.195212,1404.410306,0.138068,1.110125,0.0,0.999988,0.000019,2.570014e+46,-1.0,-0.25
2,17,5.0,0.108845,0.108845,-1.146222e-13,0.0,0.0,0.0,-0.732385,-4.193214,...,204.470239,1383.919005,0.136990,1.114414,0.0,0.999986,0.000021,2.688820e+46,-1.0,-0.25
3,18,5.0,0.113102,0.113102,-1.239873e-13,0.0,0.0,0.0,-0.703268,-4.142182,...,201.678757,1363.203146,0.136310,1.118595,0.0,0.999985,0.000023,2.809798e+46,-1.0,-0.25
4,19,5.0,0.117363,0.117363,-1.327007e-13,0.0,0.0,0.0,-0.674348,-4.091646,...,198.795191,1342.099109,0.135199,1.122682,0.0,0.999984,0.000024,2.933236e+46,-1.0,-0.25


In [ ]:
grid.get_directory_path('full_isos')